In [ ]:
!/opt/conda/bin/python3.7 -m pip install --upgrade pip
! pip install -q efficientnet

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
import os
import re
import math
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from kaggle_datasets import KaggleDatasets
import efficientnet.tfkeras as efn
import cv2

In [ ]:
GCS_DS_PATH = KaggleDatasets().get_gcs_path('ranzcr-clip-catheter-line-classification')
GCS_DS_PATH

In [ ]:
train_df = pd.read_csv(GCS_DS_PATH+"/train.csv")
train_df.index = train_df["StudyInstanceUID"]
del train_df["StudyInstanceUID"]

train_annot_df = pd.read_csv(GCS_DS_PATH+"/train_annotations.csv")
train_annot_df.index = train_annot_df["StudyInstanceUID"]
del train_annot_df["StudyInstanceUID"]

In [ ]:
train_df.head()

In [ ]:
train_annot_df.head()

In [ ]:
classes = list(train_df.columns[:-1])
classes_normal= [name for name in classes[:-1] if name.split(" - ")[1] == "Normal"]
classes_abnormal= [name for name in classes[:-1] if name.split(" - ")[1] == "Abnormal"]
classes_borderline = [name for name in classes[:-1] if name.split(" - ")[1] == "Borderline"]
classes_count = train_df[classes].sum(axis = 0)
num_classes = len(classes_count)


print("Number of Classes: {}".format(num_classes))
classes_count

In [ ]:
class_weights = {}
ls = list(classes_count.values)
tot_samples = sum(ls)

for i in range(num_classes):
    class_weights[i] = tot_samples/(num_classes*ls[i])

class_weights

In [ ]:
patient_ids = train_df["PatientID"].unique()
patientwise_count = train_df['PatientID'].value_counts()
num_patients = len(patientwise_count)
print("Number of patients: ",num_patients)
patientwise_count

In [ ]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver() 
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy() 

print("REPLICAS: ", strategy.num_replicas_in_sync)

In [ ]:
IMAGE_SIZE = [600,600]
HEIGHT,WIDTH = IMAGE_SIZE[0],IMAGE_SIZE[1]
CHANNELS = 3

AUTO = tf.data.experimental.AUTOTUNE

FILENAMES = tf.io.gfile.glob(GCS_DS_PATH + '/train_tfrecords/*.tfrec')
TRAINING_FILENAMES = FILENAMES[:-2]
VALIDATION_FILENAMES = FILENAMES[-2:]
TEST_FILENAMES = tf.io.gfile.glob(GCS_DS_PATH + '/test_tfrecords/*.tfrec')

In [ ]:
def decode_image(image_data):
    image = tf.image.decode_jpeg(image_data, channels=3)
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.image.resize(image, [*IMAGE_SIZE])
    return image


def read_labeled_tfrecord(example):
    LABELED_TFREC_FORMAT = {
        "StudyInstanceUID"           : tf.io.FixedLenFeature([], tf.string),
        "image"                      : tf.io.FixedLenFeature([], tf.string),
        "ETT - Abnormal"             : tf.io.FixedLenFeature([], tf.int64), 
        "ETT - Borderline"           : tf.io.FixedLenFeature([], tf.int64), 
        "ETT - Normal"               : tf.io.FixedLenFeature([], tf.int64), 
        "NGT - Abnormal"             : tf.io.FixedLenFeature([], tf.int64), 
        "NGT - Borderline"           : tf.io.FixedLenFeature([], tf.int64), 
        "NGT - Incompletely Imaged"  : tf.io.FixedLenFeature([], tf.int64), 
        "NGT - Normal"               : tf.io.FixedLenFeature([], tf.int64), 
        "CVC - Abnormal"             : tf.io.FixedLenFeature([], tf.int64), 
        "CVC - Borderline"           : tf.io.FixedLenFeature([], tf.int64), 
        "CVC - Normal"               : tf.io.FixedLenFeature([], tf.int64), 
        "Swan Ganz Catheter Present" : tf.io.FixedLenFeature([], tf.int64),
    }
    
    example = tf.io.parse_single_example(example, LABELED_TFREC_FORMAT)
    image = decode_image(example['image'])
    label = [example['ETT - Abnormal'],
                 example['ETT - Borderline'],
                 example['ETT - Normal'],
                 example['NGT - Abnormal'],
                 example['NGT - Borderline'],
                 example['NGT - Incompletely Imaged'],
                 example['NGT - Normal'],
                 example['CVC - Abnormal'],
                 example['CVC - Borderline'],
                 example['CVC - Normal'],
                 example['Swan Ganz Catheter Present']]
    label = [tf.cast(i,tf.float32) for i in label]
    return image, label


def read_unlabeled_tfrecord(example):
    UNLABELED_TFREC_FORMAT = {
        "StudyInstanceUID"           : tf.io.FixedLenFeature([], tf.string),
        "image"                      : tf.io.FixedLenFeature([], tf.string),
    }
    example = tf.io.parse_single_example(example, UNLABELED_TFREC_FORMAT)
    image = decode_image(example['image'])
    idnum = example['StudyInstanceUID']
    return image, idnum

def load_dataset(filenames, labeled=True, ordered=False):
    ignore_order = tf.data.Options()
    if not ordered:
        ignore_order.experimental_deterministic = False 
    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads=AUTO) 
    dataset = dataset.with_options(ignore_order)
    dataset = dataset.map(read_labeled_tfrecord if labeled else read_unlabeled_tfrecord, num_parallel_calls=AUTO)
    return dataset

In [ ]:
import tensorflow.experimental.numpy as tnp

@tf.function
def data_augment(image, label):
    p_spatial = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
    p_rotate = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
    p_pixel_1 = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
    p_pixel_2 = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
    p_pixel_3 = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
    p_crop = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
    
    if p_pixel_1 >= .4:
        image = tf.image.random_saturation(image, lower=.7, upper=1.3)
    if p_pixel_2 >= .4:
        image = tf.image.random_contrast(image, lower=.8, upper=1.2)
    if p_pixel_3 >= .4:
        image = tf.image.random_brightness(image, max_delta=.1)
        
    
    if p_crop > .7:
        if p_crop > .9:
            image = tf.image.central_crop(image, central_fraction=.7)
        elif p_crop > .8:
            image = tf.image.central_crop(image, central_fraction=.8)
        else:
            image = tf.image.central_crop(image, central_fraction=.9)
    elif p_crop > .4:
        crop_size = tf.random.uniform([], int(HEIGHT*.8), HEIGHT, dtype=tf.int32)
        image = tf.image.random_crop(image, size=[crop_size, crop_size, CHANNELS])
        
    image = tf.image.resize(image, [*IMAGE_SIZE])
    image = tf.image.random_flip_left_right(image)
    return image,label   

def get_training_dataset():
    dataset = load_dataset(TRAINING_FILENAMES, labeled=True)
    dataset = dataset.map(data_augment, num_parallel_calls=AUTO)
    dataset = dataset.repeat() 
    dataset = dataset.shuffle(2048)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTO)
    return dataset

def get_validation_dataset(ordered=False):
    dataset = load_dataset(VALIDATION_FILENAMES, labeled=True, ordered=ordered)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.cache()
    dataset = dataset.prefetch(AUTO)
    return dataset

def get_test_dataset(ordered=False):
    dataset = load_dataset(TEST_FILENAMES, labeled=False, ordered=ordered)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTO)
    return dataset

def count_data_items(filenames):
    n = [int(re.compile(r"-([0-9]*)\.").search(filename).group(1)) for filename in filenames]
    return np.sum(n)

NUM_TRAINING_IMAGES = count_data_items(TRAINING_FILENAMES)
NUM_VALIDATION_IMAGES = count_data_items(VALIDATION_FILENAMES)
NUM_TEST_IMAGES = count_data_items(TEST_FILENAMES)
print('Dataset: {} training images, {} validation images, {} unlabeled test images'.format(NUM_TRAINING_IMAGES, NUM_VALIDATION_IMAGES, NUM_TEST_IMAGES))

In [ ]:
BATCH_SIZE = 16 * strategy.num_replicas_in_sync

train_ds = get_training_dataset()
valid_ds = get_validation_dataset()
test_ds = get_test_dataset()

print("Training:", train_ds)
print("Validation:",valid_ds)
print("Test:", test_ds)

In [ ]:
print("Training data shapes:")
for image, label in train_ds.take(3):
    print(image.numpy().shape, label.numpy().shape)
print("Training data label examples:", label.numpy())

In [ ]:
print("Test data shapes:")
for image, idnum in test_ds.take(3):
    print(image.numpy().shape, idnum.numpy().shape)

In [ ]:
def create_model():
    efficient_net = {
        0 : efn.EfficientNetB0(weights="noisy-student",include_top=False ,input_shape=[*IMAGE_SIZE, 3]),
        1 : efn.EfficientNetB1(weights="noisy-student",include_top=False ,input_shape=[*IMAGE_SIZE, 3]),
        2 : efn.EfficientNetB2(weights="noisy-student",include_top=False ,input_shape=[*IMAGE_SIZE, 3]),
        3 : efn.EfficientNetB3(weights="noisy-student",include_top=False ,input_shape=[*IMAGE_SIZE, 3]),
        4 : efn.EfficientNetB4(weights="noisy-student",include_top=False ,input_shape=[*IMAGE_SIZE, 3]),
        5 : efn.EfficientNetB5(weights="noisy-student",include_top=False ,input_shape=[*IMAGE_SIZE, 3]),
        6 : efn.EfficientNetB6(weights="noisy-student",include_top=False ,input_shape=[*IMAGE_SIZE, 3]),
        7 : efn.EfficientNetB7(weights="noisy-student",include_top=False ,input_shape=[*IMAGE_SIZE, 3]),
        8 : tf.keras.applications.Xception(include_top=False, weights='imagenet',input_shape=[*IMAGE_SIZE, 3]),
        9 : tf.keras.applications.ResNet50(include_top=False, weights='imagenet',input_shape=[*IMAGE_SIZE, 3]),
        10: tf.keras.applications.ResNet101(include_top=False, weights='imagenet',input_shape=[*IMAGE_SIZE, 3]),
        11: tf.keras.applications.ResNet152(include_top=False, weights='imagenet',input_shape=[*IMAGE_SIZE, 3])
    }
    
    ls = [7]
    output = {}
    inputs = tf.keras.Input(shape=(*IMAGE_SIZE, 3))
    for i in ls:
        pretrained_model = efficient_net[i]
        x = pretrained_model(inputs)
        x = tf.keras.layers.GlobalAveragePooling2D()(x)
        x = tf.keras.layers.Dense(512) (x)
        x = tf.keras.layers.LeakyReLU(alpha=0.2) (x)
        x = tf.keras.layers.Dropout(0.3) (x)
        x = tf.keras.layers.Dense(256) (x)
        x = tf.keras.layers.LeakyReLU(alpha=0.2) (x)
        x = tf.keras.layers.GaussianDropout(0.4) (x)
        output[i] = tf.keras.layers.Dense(num_classes,activation="sigmoid", dtype='float32')(x)
        
    if len(ls)>1:
        outputs = tf.keras.layers.average(list(output.values()))
    else:
        outputs = list(output.values())[0]
        
    model = tf.keras.Model(inputs, outputs)
    
    return model

In [ ]:
def compile_model(model, lr=0.0001):
    
    optimizer = tf.keras.optimizers.Adam(lr=lr)
    
    loss = tf.keras.losses.BinaryCrossentropy(label_smoothing=0.05)
        
    metrics = [
        tf.keras.metrics.AUC(name='auc',multi_label=True)
    ]

    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

    return model

In [ ]:
def create_callbacks(model_save_path):
    
    
    if not os.path.exists(model_save_path):
        os.makedirs(model_save_path)
    
    cpk_path = f'{model_save_path}/model.h5'

    checkpoint = tf.keras.callbacks.ModelCheckpoint(
        filepath=cpk_path,
        monitor='val_auc',
        mode='max',
        save_best_only=True,
        verbose=1
    )

    reducelr = tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_auc',
        mode='max',
        factor=0.1,
        patience=3,
        verbose=0
    )

    earlystop = tf.keras.callbacks.EarlyStopping(
        monitor='val_auc',
        mode='max',
        patience=10, 
        verbose=1
    )
    
    callbacks = [checkpoint, reducelr, earlystop]
    
    return callbacks

In [ ]:
EPOCHS= 30
VERBOSE =1
MODEL_PATH = '.'
STEPS_PER_EPOCH = NUM_TRAINING_IMAGES//(BATCH_SIZE)

tf.keras.backend.clear_session()
with strategy.scope():
    model = create_model()
    model = compile_model(model, lr=0.0001)
   
    callbacks = create_callbacks(MODEL_PATH)
    
    history = model.fit(train_ds, 
                        epochs=EPOCHS,
                        callbacks=callbacks,
                        validation_data = valid_ds,
                        steps_per_epoch = STEPS_PER_EPOCH,
                        verbose=VERBOSE)

In [ ]:
model.summary()

In [ ]:
model = tf.keras.models.load_model("./model.h5")

In [ ]:
test_ids=[]
test_pred = []
for batch in test_ds:
    images,ids_batch = batch
    pred_batch = model.predict(images)
    for i,ids in enumerate(ids_batch):
        test_ids.append(ids)
        test_pred.append(pred_batch[i])

test_ids = [np.array(i).astype("str").tolist() for i in test_ids]
test_df = pd.DataFrame(test_pred,index=test_ids,columns=classes)
test_df.index.name = "StudyInstanceUID"
test_df.to_csv("./submission.csv")